In [2]:
def OpticalFlow(im,grad_x,grad_y,grad_t,win_size,thresh):
    import numpy as np
    h,w=im.shape
    u=np.zeros((h,w))
    v=np.zeros((h,w))
    gap=win_size[0]//2
    
    #Calculating sum(grad_x^2),sum(grad_y^2),sum(grad_x*grad_y),sum(grad_x*grad_t),sum(grad_t*grad_y)
    #at every pixel for a window of 3x3
    for i in range(0,h,gap+1):
        for j in range(0,w,gap+1):
            if i-gap<0 or j-gap<0 or i+gap==h or j+gap==w:
                pass
            else:
                grad_x_sq=np.sum(grad_x[i-gap:i+gap+1,j-gap:j+gap+1]**2)
                grad_y_sq=np.sum(grad_y[i-gap:i+gap+1,j-gap:j+gap+1]**2)
                grad_xy=np.sum(np.multiply(grad_x[i-gap:i+gap+1,j-gap:j+gap+1],grad_y[i-gap:i+gap+1,j-gap:j+gap+1]))
                grad_xt=np.sum(np.multiply(grad_x[i-gap:i+gap+1,j-gap:j+gap+1],grad_t[i-gap:i+gap+1,j-gap:j+gap+1]))
                grad_yt=np.sum(np.multiply(grad_y[i-gap:i+gap+1,j-gap:j+gap+1],grad_t[i-gap:i+gap+1,j-gap:j+gap+1]))            
                #print(grad_x_sq,grad_y_sq,grad_xy,grad_xt,grad_yt)
                
                #Forming the matrix
                A = np.array([ [ grad_x_sq,grad_xy], [grad_xy, grad_y_sq] ])
                b = np.array([-grad_xt, -grad_yt])
                #print(A)
                # if threshold τ is larger than the smallest eigenvalue of A'A:
                U,D,V_T = np.linalg.svd(A.T.dot(A))
                #print(np.min(D))
                if np.min(D)<thresh:
                    u[i-gap:i+gap+1,j-gap:j+gap+1]=0
                    v[i-gap:i+gap+1,j-gap:j+gap+1]=0                

                else:
                    o_flow = np.linalg.inv(A.T.dot(A)).dot(A.T).dot(b)
                    #o_flow = np.linalg.inv(A).dot(b)
                    u[i-gap:i+gap,j-gap:j+gap]=o_flow[0]
                    v[i-gap:i+gap,j-gap:j+gap]=o_flow[1]
    return(u,v)
def Calc_Flow(folder,img1,img2,window=(3,3),threshold=0.01):
    import numpy as np
    import cv2
    import scipy
    from scipy import signal
    from scipy.signal import convolve2d
 
    #Computing Ix,Iy,It at every point on the img I1
    Kernel_gradx=np.array([[-1,1],[-1,1]])
    Kernel_grady=np.array([[1,1],[-1,-1]])
    Kernel_gradt=np.array([[1,1],[1,1]])*.25
    #grad_x=convolution2d(im1,Kernel_gradx)
    grad_x=convolve2d(img1, Kernel_gradx, boundary='symm', mode='same')
    grad_y=convolve2d(img1, Kernel_grady, boundary='symm', mode='same')
    grad_t=convolve2d(img1, -Kernel_gradt, boundary='symm', mode='same') +
           convolve2d(img2, Kernel_gradt, boundary='symm', mode='same')
    u,v=OpticalFlow(img1,grad_x,grad_y,grad_t,window,threshold)
    return u,v

In [3]:
def FlowMaps(folder,img1,u,v,granularity):
    import cv2
    import numpy as np
    h,w=img1.shape
    flow_map=np.zeros((h,w))
    im=img1
    #Visualization
    for y in range(h):
            for x in range(w):
                if y % granularity == 0 and x % granularity == 0:
                    dx = int(u[y, x] )
                    dy = int(v[y, x])
                    cv2.arrowedLine(im, (x, y), (x + dx, y + dy), (255,255,255))
                    cv2.arrowedLine(flow_map, (x, y), (x + dx, y + dy), (255,255,255))
    cv2.imwrite('./'+folder+'/FlowOnImg'+str(granularity)+'.png',im)
    cv2.imwrite('./'+folder+'/FlowMap'+str(granularity)+'.png',flow_map)
    return im, flow_map

In [8]:
if __name__ == "__main__":
    import numpy as np
    import cv2
    import scipy
    from scipy import signal
    from scipy.signal import convolve2d 
    
    folder=['Army','Backyard','Basketball','Dumptruck','Evergreen','Grove','Mequon','Schefflera','Teddy','Urban','Wooden','Yosemite']
    granularity=6
    for i in folder:
        img1=cv2.imread('./'+i+'/frame10.png')
        img2=cv2.imread('./'+i+'/frame11.png')
        #converting to gray_scale
        img1=cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
        img2=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
        u,v=Calc_Flow(i,img1,img2)
        im,f_im=FlowMaps(i,img1,u,v,granularity)